# Mixed View

## Set correct directory

In [1]:
cd ..

/bptk-py


## Import libraries

In [2]:
import pandas as pd
import csv
import time
import pickle
from elasticsearch import Elasticsearch
import os.path
from src.config.conf import width,height
from src.setup import setup_model
from ipywidgets import widgets
from ipywidgets import IntSlider

## Batch View

In [4]:
# simple query just unpickles the data
data_file_object = open ("csv/data.pickle",'rb')
data=pickle.load(data_file_object)
data

In [6]:
time_file_object = open("csv/sim_time.pickle",'rb')  
current_time = pickle.load(time_file_object)
current_time

## Real Time View

## Mixed Query Using Batch and Real Time Views